<img src="../../../thu_sigs_logo.png" alt="清华深研院-横" style="zoom:50%;" />

In [1]:
#| default_exp digital_processing_of_speech_signals.hidden_markov_model

In [2]:
#|hide
# autoreload
%load_ext autoreload
%autoreload 2
%load_ext rich

## 代码与文档格式说明

本文档使用Jupyter Notebook编写，所以同时包括了实验文档和实验代码。

本次实验项目采用了 Quarto + nbdev 的系统来发布Jupyter Notebook, 因而我们的实验文档导出为pdf和html格式可以进行阅读，而我们的代码也导出为python模块形式，可以作为代码库被其他项目使用。

我们这样做的好处是，避免单独管理一堆 .py 文件，防止代码冗余和同步混乱，py文件和pdf文件都是从.ipynb文件导出的，可以保证实验文档和代码的一致性。

### 文档阅读说明 {#sec:reading_notes}

::: {.callout-important}

本文档支持多个格式，包括ipynb, html, docx, pdf, md 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读。

但是由于本文档具有一定的交互性，我们优先建议您打开我们构建的[在线文档网站](https://thu-coursework-machine-learning-for-big-data-docs.vercel.app/) 或者 [github pages](https://open-book-studio.github.io/THU-Coursework-Machine-Learning-for-Big-Data/)进行阅读，或者使用浏览器打开作业提交的文件夹中`html`格式的文档本地打开进行阅读。

其次是建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

VSCode或者浏览器阅读的ipynb和html文档都可以显示交互性图表、数据，这样比pdf文件阅读体验更佳。

:::

### 代码使用说明

https://github.com/Open-Book-Studio/THU-Coursework-Machine-Learning-for-Big-Data.git 是我们本次大数据机器学习课程实验的代码仓库地址。

我们的实验代码都是通过 jupyter notebook 进行编写，并通过 nbdev 进行自动化构建pypi库。
复现我们实验代码时，建议使用支持jupyter notebook的IDE（如VSCode）运行我们的 `ipynb文件`。

::: {.callout-note}
可以通过以下命令安装我们实验的代码：

```shell
git clone https://github.com/Open-Book-Studio/THU-Coursework-Machine-Learning-for-Big-Data.git
cd THU-Coursework-Machine-Learning-for-Big-Data
pip install -e .
```
我们的代码导出为了python模块形式，如果你需要在别的项目中调用我们的代码，可以通过以下命令导入：
```python
from thu_big_data_ml.digital_processing_of_speech_signals.hidden_markov_model import *
```
:::

这次作业中，我开发的我们课题组的基础依赖库[ScholarlyInfrastructure](https://github.com/THU-CVML/ScholarlyInfrastructure)也相应地进行了代码更新。这个库对标`fastcore`库，对AI科研经常会用到的一些基础性地、和Python语言的表达力有关的代码进行了整理，比如PyTorch模型检查、清晰的日志、实验参数管理、异常处理、argmax自动函数优化等。接下来我们也会用到这个项目中的一些代码来完成本次作业。

::: {.callout-tip}

通过以下命令安装 ScholarlyInfrastructure

```shell
git clone https://github.com/THU-CVML/ScholarlyInfrastructure.git
cd ScholarlyInfrastructure
pip install -e .
```

```python
from scholarly_infrastructure import *
```

以上的代码库都已经开源在github和pypi，欢迎各位同学、老师们提出宝贵意见，或者加入我们的开发一起完善，构建更加优质的科研工具。
:::



In [3]:
from scholarly_infrastructure.logging.nucleus import logger, print
from thu_big_data_ml.help import plt, pio
import treescope
treescope.active_autovisualizer.set_globally(treescope.ArrayAutovisualizer())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 from scholarly_infrastructure.logging.nucleus import logger, print                           │
│ ❱ 2 from thu_big_data_ml.help import plt, pio                                                    │
│   3 import treescope                                                                             │
│   4 treescope.active_autovisualizer.set_globally(treescope.ArrayAutovisualizer())                │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'thu_big_data_ml'

In [5]:
#| hide 
import nbdev; nbdev.nbdev_export()

## 1. 基本任务：查找阅读文献资料并回答问题

### 1） 查找并阅读至少 3 篇有关 HMM 模型的参考文献资料

> 其中：至少 1 篇为经过同行评审的正式发表的英文论文（会议或期刊均可）；不得多于 1 篇为知乎、博客、百科等非正式发表的科普性阅读资料或讲义。（10 分）

根据老师给我们的要求，我们需要找到至少 3 篇有关 HMM 模型的参考文献资料。
我们查找资料的目的是为了学习理解 HMM 模型的基本原理和应用，深入地回答老师问我们的几个问题以及我们自己的问题。

最终我选择的文献包括

- 李航的《机器学习方法》 [@LiHang_2022]
    - 这个是正式出版发表的中文书籍(教材课本)，经过了出版社编辑的评审，但是不一定是经过了论文的同行评审。
    - 这个书是机器学习领域的经典教材，数学推导较为详细，叙述较为清晰，适合深入学习。

- 老师作业文档里面建议我们阅读的 Tutorial [@Rabiner_1989]
    - 这是经过同行评审的英语论文。发表在 Proceedings of the IEEE 上。
    - 唯一作者 LAWRENCE R. RABINER 是IEEE Fellow， H-index 39。
    
- 自然语言处理方向使用到HMM的最新前沿论文，具体方向不限，我选择了 "Adaptable Logical Control for Large Language Models" [@Zhang_2024]

    - 这是一篇经过同行评审的英文论文。发表在 AI三大顶会之一的 NeurIPS 上，是2024年最新的文章。
    - 第一作者是 Zhang Honghua，五个作者都来自于UCLA。
    - 论文核心贡献在于使用HMM来实现大模型的可控输出，我觉得这是个很有意思的研究问题。

除了这三篇论文，我还可能在回答问题过程中引用一些其他资料或者论文。


::: {.callout-note}
关于“Proceedings of the IEEE”的一些问题。我们通过搜索引擎和AI查询，回答我们的一些问题。

- “Proceedings”是什么意思？和Transaction、Conference等词语的关系是什么？
    - "Proceedings（会议录/会议论文集）" 是指特定学术会议结束后出版的论文集合，通常包含会议中发表的论文、演讲、讨论和报告，这些论文可能未经同行评审。
        - 追问：那 Proceedings of the IEEE 是不是经过了同行评审？
        - 追问：这个英语单词明明原意是“进度”“进展”，或者“诉讼程序”，为什么在IEEE里面引申成了“会议集”，听起来好奇怪啊？
        - 追问：那么 Proceedings 到底是 会议还是期刊呢？
    - "Transactions（学报）" 是指一系列定期出版的学术期刊，它们通常包含经过严格同行评审的论文，强调期刊的连续性和系统性，常见于工程和技术领域。
        - 追问：这个英语单词原本的意思是交易，或者我们学数据库课的时候，数据库里面的“事务”，为什么也有学报的意思，英语上是怎么引申而来的？
        - 追问：经常听别的专业的人说有“清华大学学报”这种高校的学报可以发论文，但是我们计算机科学和人工智能领域我好像没见过啊，是不是对于我们这个领域来说这些学报是很水的？
    - "Conference（会议）" 是指学术会议本身，是学者们聚集在一起交流研究成果、讨论学术问题的活动，可以是国际的、国内的，或特定领域的专业会议。
        - 追问：其他学科的人为什么会认为“会议很水，比期刊弱多了”，而我们CS学科却认为“会议”才是最重要的？
    - "Workshop（研讨会/工作坊）" 是指规模较小、更加非正式的学术会议，通常专注于某个特定的研究领域或问题，强调互动和讨论，可能包括讲座、小组讨论和实践环节。
        - 追问：为什么很多师兄师姐会提到，“Workshop很水，千万别发，也不要看”？从字面意义来看似乎没有说Workshop很垃圾啊？如果我发一篇Workshop是不是毕不了业？

    - "Journal（期刊）" 是指定期出版的学术出版物，包含经过同行评审的学术论文，可以是综合性的，也可以是特定学科或领域的专业性期刊。

- “Proceedings of the IEEE”影响力如何？是不是 CCF 和 TH-CPL A类期刊会议？如果我发一篇这个，能否毕业？
    - 根据我的查询，我们发现，[CCF](https://ccf.atom.im/)和[清华分级TH-CPL](https://numbda.cs.tsinghua.edu.cn/~yuwj/TH-CPL.pdf)都一致认为，这是重要的“综合与交叉（Interdisciplinary）”A类期刊！
    - 这个结果让我震惊，因为要想发这里面的文章，似乎我只需要写Tutorial，不需要提出自己的方法达到SOTA！这是另一个赛道！
    - 而且Tutorial的影响力很大，这个期刊的年平均引用量高达46.4。

- 老师提到的语音领域相关会议、期刊有TASLP，ICASSP，INTERSPEECH。这些期刊会议是几类CCF和TH-CPL呢？如果我发这些能毕业吗？
    - 根据搜索结果，这三个分别是 CCF-B期刊, CCF-B和CCF-C会议。
    - 但是对于[清华分级TH-CPL](https://numbda.cs.tsinghua.edu.cn/~yuwj/TH-CPL.pdf)来说，TASLP升级为A类，ICASSP仍为B类，INTERSPEECH未上榜。
:::



### 2） 以标准参考文献的格式列出所阅读的参考文献列表，并在回答以下问题时给出明确的引用。（5 分）

根据老师提供的 ICASSP 论文的要求：

> - REFERENCES: List and number all bibliographical references at the end of the paper. The 
> references can be numbered in alphabetic order or in order of appearance in the 
> document. When referring to them in the text, type the corresponding reference 
> number in square brackets as shown at the end of this sentence [2].
> 
> - The end of the document should include a list of references containing information similar 
> to the following examples (Pay attention to all format details, including but not limited to 
> authors’ name list, lower/upper-cases, punctuations, etc.):
>
>   - [1] D. E. Ingalls, "Image Processing for Experts," IEEE Transactions on Acoustics, Speech, and Signal Processing (TASSP), vol. TASSP-36, pp. 1932-1948, 1988.



论文的引用(citation)包括 in text citation  和 bibliography 。根据上面的要求，in text citation 应该是"[1]" 而非"(author, date)"的格式。而bibliography排序是按照字母顺序或者文档中出现的先后顺序。从示例来看，这很可能是 IEEE 格式，而且ICASSP是IEEE旗下的会议。

IEEE引用格式可以被`CSL` (Citation Style Language)语言形式化定义，参考文件[ieee.csl](https://github.com/citation-style-language/styles/blob/master/ieee.csl)。`CSL`语言广泛地被使用在 zotero, latex, typst 等软件系统中。

而参考文献本身一般使用 `BibLaTeX` 格式来进行记录，这一格式也已经被广泛使用。需要注意的是，`BibLaTeX` 不是 `BibTeX`格式，两者有区别，参考https://pandoc.org/MANUAL.html#citations。

本文档使用 quarto 科学文档系统进行撰写，采用的是 Pandoc Markdown 格式来在jupyter notebook中引入参考文献，参考 https://quarto.org/docs/authoring/citations.html 。quarto也支持CSL和BibLaTeX。需要注意的是，latex中`\cite`与`\ref`不同，前者是引用文献，后者是引用图表、公式、章节等。但是在 pandoc 引用格式中，这两者都是使用`@key`的方式来 in text citation，而被交叉引用的目标使用 `{#key}`  来标注。

我们在 `ipynb` 中的第一个raw block中指定了对应的 `CSL` 样式文件 `ieee.csl`，并在 `BibLaTeX` 格式的参考文献文件 `references.bib` 中，然后在正文中使用`[@citation_key1, @citation_key2]`的格式来进行In Text Citation。



如 ICASSP 的要求，我们把所有的参考文献列表放在了文末 @sec-ref-list。

::: {.callout-warning}
如果您是通过jupyter notebook而非通过文档网站阅读我们的文档 (见@sec:reading_notes的说明)，您可能无法看到自动生成的参考文献，所以我们也把刚才我们找的三篇文献以文本形式在此列出。
<!-- TODO -->
[1] 李航, 机器学习方法. <a href="https://book.douban.com/press/2562">清华大学出版社</a>    <br>                                                      <span class="pl">出版年:</span> 2022-3, 2022. Accessed: Dec. 07, 2024. [Online]. Available: https://book.douban.com/subject/35884788/

[2] L. R. Rabiner, “A tutorial on hidden Markov models and selected applications in speech recognition,” Proceedings of the IEEE, vol. 77, no. 2, pp. 257–286, Feb. 1989, doi: 10.1109/5.18626.

[3] H. Zhang, P.-N. Kung, M. Yoshida, G. V. den Broeck, and N. Peng, “Adaptable Logical Control for Large Language Models,” presented at the The Thirty-eighth Annual Conference on Neural Information Processing Systems, Nov. 2024. Accessed: Dec. 08, 2024. [Online]. Available: https://openreview.net/forum?id=58X9v92zRd&referrer=%5Bthe%20profile%20of%20Honghua%20Zhang%5D(%2Fprofile%3Fid%3D~Honghua_Zhang1)

:::


### 3）什么是马尔可夫过程？一阶马尔可夫过程的特点是什么？（可结合图）对一阶马尔可夫过程及一阶马尔可夫系统进行明确的形式化定义。（3 分）

#### 什么是马尔可夫过程?

从词语本身的定义来看，马儿可夫过程(Markov Process)是指具备了马尔科夫性质的随机过程 [@LiHang_2022]。所以要理解什么是马尔可夫过程，无非就是要理解清楚这两个子概念：马尔可夫性质和随机过程。

1. 首先什么是随机过程呢？这个我在之前的课上确实没有学过，所以我们这里认真的把这个概念梳理一下。

李航在第 19 章准备对马尔科夫链进行定义的时候，先对随机过程进行了一个较为严谨的定义 [@LiHang_2022]：

> 考虑一个随机变量的序列$X=\left \{X_0,X_1,...,X_t,...\right\}$，这里$X_t$表示时刻t的随机变量，t=0,1,2,...。每个随机变量$X_t$(t=0,1,2,...)的取值集合相同，称为状态空间，表示为S。随机变量可以是离散的，也可以是连续的。以上随机变量的序列构成**随机过程（stochastic process）**。

可以看到李航的定义的关键点在于两个，首先随机过程是随机向量(序列)构成的，其次就是这个向量里面的每一个随机变量共享一个一样的状态空间S。

李航这样定义的意思其实就是想要建模一个随时间变化分布会变化的随机变量，可能取值范围倒是不会变，毕竟还是同一个东西，像是股票的价格，天气的状态，但毕竟分布变化了，所以就不是同一个随机变量了，而是得做一列随机变量序列，叫做随机过程。

随机过程的英文也可以是 Random Process, t=0,1,2,...叫做索引集合(index set)，虽然一般index set都是时间，但是没有限制 [@Stochastic_process_2024]。相邻时间点的两个随机变量的差$X_{t+1}-X_t$ 也称为增量(increment) [@Stochastic_process_2024]。李航的定义只考虑序列，但是index set不一定是有限或者可数无穷的集合，也可以是连续的实数，后者叫做continuous-time stochastic processes，而前者叫做discrete-time stochastic processes; 如果进一步index set不仅离散而且就是整数集合或者子集，那么这个随机过程还可以叫做随机序列(random sequence) [@Stochastic_process_2024]。


2. 那么什么是马尔可夫性质呢？

> 假设在时刻0的随机变量$X_0$遵循概率分布$P(X_0)=\pi_0$，称为初始状态分布。在某个时刻$t \geq 1$的随机变量$X_t$与前一个时刻的随机变量$X_{t-1}$之间有条件分布$P(X_t|X_{t-1})$；如果$X_t$只依赖于$X_{t-1}$，而不依赖于过去的随机变量$\{X_0, X_1, \cdots, X_{t-2}\}$，这一性质称为马尔可夫性，即  
>
>$$
>P(X_t|X_0, X_1, \cdots, X_{t-1}) = P(X_t|X_{t-1}), \quad t=1,2,\cdots
>$$ {#eq:markov}
>



3. 什么是马尔可夫过程？


>具有马尔可夫性的随机序列$X = \{X_0, X_1, \cdots, X_t, \cdots\}$称为马尔可夫链(Markov chain)或马尔可夫过程(Markov process)。条件概率分布$P(X_t|X_{t-1})$称为马尔可夫链的转移概率分布。转移概率分布决定了马尔可夫链的特性。



#### 什么是一阶马尔可夫过程及一阶马尔可夫系统？

#### 一阶马尔可夫过程的特点是什么？

搞明白了什么是一阶马尔可夫过程。

## 任务二：基于HMM的强制对齐实验


## 参考文献 References {#sec-ref-list}

::: {#refs}
:::

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()